In this lecture, we learn about linked structures, a family of data structures that can be visualized with diagrams of boxes with arrows between them.  That style is a natural match with recursion, which we have been studying so far mostly in the context of search problems.  Linked structures are related to the concept of pointers in some non-Python languages, and recursion and pointers together have been [described](https://www.joelonsoftware.com/2005/12/29/the-perils-of-javaschools-2/) as the defining difficult concepts of programming or computer science.  If nothing else, practicing these concepts will prepare you for the interview questions that the big-name software companies like to use!  It doesn't hurt that these ideas also form the bedrock for most kinds of algorithmically challenging coding.

# A Puzzle with Unintended Sharing in Python Lists

The code for this lecture begins with a program demonstrating a problem that some students ran into in a lab near the beginning of the semester in 6.009.  Here's an easy way to initialize a list of lists, where each element of the overall list is an empty list.

In [1]:
n = 5
ls = [[]] * n

For the example of `n = 3`, the programmer might be thinking of this code as creating a list that we draw like:

![Aliasing in action](aliasing.png)

In other words, the programmer is expecting that each element of the main list is referencing a *separate* empty list.  Unfortunately, when we modify all of the constituent lists in most any way, we discover how the naïve model is misguided!  The reason is that the *real* effect of the code above is to create *one* empty list and then stash its references in all cells of the main list.  In other words, the true situation looks like:

![Aliasing averted](noaliasing.png)

A variety of different short code snippets could lead to the first drawing instead.  One of them uses a list comprehension:

In [2]:
ls = [[] for i in range(n)]

The important thing here is that the `[]` expression gets reevaluated for every value of `i`, creating a new list!

The possibility for several arrows (called *pointers* or *references* in different contexts) to go to the same box is called *aliasing*, and it's a persistent source of complexity in programming.  You might be wondering why such a confusing feature should be  tolerated in a programming language.  It turns out that aliasing can enable some satisfyingly elegant programming styles, including one that we focus on in the rest of the lecture, which allows us to maintain *several closely related copies of a data structure, reusing space across them*.  The important thing is that we will avoid in-place updates and leave each data node alone after we create it, following part of the philosophy of *functional programming*.

# An Interlude on Estimating Running Time of Python Programs

In covering some new styles of data-structure representation, we'll want a way to evaluate how quickly different operations execute.  Before we get there, we need to have a clear model of how data structures are laid out in memory.  The "boxes and arrows" diagrams we just used turn out to be pretty accurate.  We can think of a Python list (array) as a sequence of cells sitting next to each other.  Think of it as the first cell of a list having address `N` in memory, such that it is easy to look up the `i`th cell, because we compute its address as `N+i`.  The underlying computer provides fast implementations for both addition and looking up a cell by address.

With that intuition, we can decompose a program operating on lists into the set of *primitive operations* it executes.  We'll consider that each primitive operation takes equal time to run, so that a good estimate of a program's running time is its count of primitive operations.

Here are some lines of Python code demonstrating primitive operations on lists.

In [3]:
l = []
# Make a new empty list.  (Initializing with multiple elements wouldn't count as primitive anymore!)
l.append(5)
l.append(7)
# Appending to a list.
v = l[1]
# Reading from a position.
l[0] = 6
# Writing to a position.

It's actually a bit nonobvious how Python can make appending take the same time as seemingly simpler operations like reading and writing cells.  What if we run out of space in memory right after the current end of the list?  The solution strays into 6.006 territory, but the curious can find information online as in [this StackOverflow question](https://stackoverflow.com/questions/5833907/repeatedly-appending-to-a-large-list-python-2-6-6).

It's important to recognize which operations are *not* primitive.  For instance, consider a list-slicing operation.

In [4]:
l1 = list(range(10))
l2 = l1[1:10]
l2

[1, 2, 3, 4, 5, 6, 7, 8, 9]

Slicing is actually implemented in terms of primitive operations, where the following code is a good guide, telling us that the slice spends a number of primitive operations equal to the size of the new list.

In [5]:
l2 = []
for i in range(1, 10):
    l2.append(l1[i])
l2

[1, 2, 3, 4, 5, 6, 7, 8, 9]

What if we want to add a new element in the middle of the list?  The one-liner solution might lull us into a false sense of running-time security.

In [6]:
l1[:5] + ['hi'] + l1[5:]

[0, 1, 2, 3, 4, 'hi', 5, 6, 7, 8, 9]

What that code *really* does is like the following, where we see again a number of primitive operations proportional to the new list length.

In [7]:
l2 = []
for i in range(5):
    l2.append(l1[i])
l2.append('hi')
for i in range(5, 10):
    l2.append(l1[i])
l2

[0, 1, 2, 3, 4, 'hi', 5, 6, 7, 8, 9]

It's also worth commenting on the *space* usage of these derived lists.  Each copy uses completely separate memory.  If we keep making lists that differ from each other in only small ways, and we need to keep all of them around, we can quickly come to occupy quite a lot of computer memory.

Let's summarize the pros and cons of Python lists.

 - **Pro**: a given list is represented compactly using sequential memory slots.
 - **Con**: many different lists that differ slightly from each other will take up lots of memory, with no sharing.
 - **Pro**: looking up or ovewriting a list cell by position is fast (a primitive operation).
 - **Pro**: adding to the end of a list is fast (a primitive operation).
 - **Con**: modifying a list or building a new one in most other ways are slow (broken into a number of primitive operations proportional to the new list length).

# Linked Lists

What are called lists in Python are called *arrays* in most other languages.  In the broader context, "list" is more likely to mean *linked list*.  What's the difference?

Like we just said, an array occupies adjacent cells of memory.
![an array](array.png)

A linked list instead involves clusters of memory cells, each of which ends in a `None` (indicating end of list) or a reference (address) of the next cluster, like so:
![a linked list](linkedlist.png)

What advantages follow from this representation?  Well, one *dis*advantage is that it takes longer to look up an element of the list by position.  With arrays (Python lists), we can look up the 1000th element of `myList` like `myList[999]`.  With linked lists, we need to follow every arrow from the beginning node (called the root) onward, until hitting element `1000`!

However, linked lists (and related data structures) often allow much faster creation of *new values based on old ones*.  If we want to add a new element to the beginning of the example list, the array way works in three steps:

![adding to the front of an array](arraycopy.png)

The copying step will take time proportional to the length of the array.  In contrast, with linked lists, we can build a new derived list in a single step that works immediately, for any addition of a new element to the beginning of a list.

![adding to the front of a linked list](listcopy.png)

We create a new list node that points to the original list root.  There was no need to change the original list.  In fact, the *original* list is still usable!  Linked structures make it very easy to build up a sequence of versions of a data structure, where versions share most of their nodes.  In this way, we not only speed up the creation of new versions, but we also save plenty of memory vs. a naïve array-based solution.

Let's look at some simple functions to create and manipulate linked lists.

In [8]:
# Basic construction of linked lists: create an empty one.
def nil():
    return None

# Create a new list that starts with x and then continues as list ls.
def cons(x, ls):
    return {"data": x,
            "next": ls}
    # Note that we are using Python dictionaries here to represent our "clusters in memory,"
    # but we will switch to a nicer implementation next.

# Is a list empty?
def isEmpty(ls):
    return ls == None

# Return the first element of a nonempty list (raises exception if empty).
def head(ls):
    return ls["data"]

# Return the part of a list after the first element (raises exception if empty).
def tail(ls):
    return ls["next"]

So here is how we can build the list diagrammed above, where, by the way, the function name `cons` stands for "construct" and was introduced in [the Lisp programming language](https://en.wikipedia.org/wiki/Lisp_(programming_language)) in the 1950s!

In [9]:
l = cons('A', cons('B', cons('C', nil())))
print(isEmpty(l))
print(head(l))
print(head(tail(l)))
print(head(tail(tail(l))))
print(isEmpty(tail(tail(tail(l)))))

False
A
B
C
True


The functions we have defined above can be considered the primitive operations that apply to linked lists.  The performance pros and cons of linked lists follow:
 - **Pro**: it's very quick to create a new linked list that is like an old one but with a new first element.
 - **Con**: it's slower to create a new linked list that is like an old one but with a new *last* element.
 - **Con**: looking up elements by numeric position is slow (proportionate to position).
 - **Pro**: many linked lists at once can *share* nodes (clusters of memory addresses)!

Let's work through some of these consequences with code examples.  Here's a simple way to compute a linked list's length, which unfortunately takes time proportional to that length.

In [10]:
def length(ls):
    len = 0

    while not isEmpty(ls):
        len += 1
        ls = tail(ls)

    return len

length(l)

3

We can realize the same approximate performance with a recursive solution.

In [11]:
def length_rec(ls):
    if isEmpty(ls):
        return 0
    else:
        return 1 + length_rec(tail(ls))

length_rec(l)

3

Here's how to build a new list that is the reverse of the original.

In [12]:
def rev(ls):
    ret = nil()

    while not isEmpty(ls):
        ret = cons(head(ls), ret)
        ls = tail(ls)

    return ret

print(head(l))
l_reversed = rev(l)
print(head(l_reversed))
print(head(l))

A
C
A


Note how the original list is left intact after the reversal.

With a little more cleverness, we can write list reversal as a recursive function.

In [13]:
def rev_rec(ls, acc=nil()):
    # "acc" stands for "accumulator," a standard term for this sort of extra parameter.

    if isEmpty(ls):
        return acc
    else:
        return rev_rec(tail(ls), cons(head(ls), acc))

print(head(l))
l_reversed = rev_rec(l)
print(head(l_reversed))
print(head(l))

A
C
A


Concatenation of lists is another classic operation, and this one is easier to do recursively.

In [14]:
def concat(ls1, ls2):
    if isEmpty(ls1):
        return ls2
    else:
        return cons(head(ls1), concat(tail(ls1), ls2))

lc = concat(l, l_reversed)
[head(lc), head(tail(lc)), head(tail(tail(lc))), head(tail(tail(tail(lc)))), head(tail(tail(tail(tail(lc)))))]

['A', 'B', 'C', 'C', 'B']

Here's a simple way to generate long lists, for testing purposes.

In [15]:
def upto(n):
    ls = nil()

    for i in range(n):
        ls = cons(i, ls)

    return ls

[head(upto(3)), head(tail(upto(3))), head(tail(tail(upto(3)))), isEmpty(tail(tail(tail(upto(3)))))]

[2, 1, 0, True]

When we try to concatenate large lists like `upto(100)`, Python tells us the maximum recursion depth is exceeded!  We can fix the problem by using an iterative implementation like so:

In [16]:
def concat_iter(ls1, ls2):
    ls1 = rev(ls1)

    while not isEmpty(ls1):
        ls2 = cons(head(ls1), ls2)
        ls1 = tail(ls1)

    return ls2

lc = concat_iter(l, l_reversed)
[head(lc), head(tail(lc)), head(tail(tail(lc))), head(tail(tail(tail(lc)))), head(tail(tail(tail(tail(lc)))))]

['A', 'B', 'C', 'C', 'B']

There is also a tricky way to define concatenation recursively, using our reversal function from before, with its optional second argument.

In [17]:
def concat_rec(ls1, ls2):
    return rev_rec(rev_rec(ls1), ls2)

lc = concat_rec(l, l_reversed)
[head(lc), head(tail(lc)), head(tail(tail(lc))), head(tail(tail(tail(lc)))), head(tail(tail(tail(tail(lc)))))]

['A', 'B', 'C', 'C', 'B']

Let's wrap up the section with a few test cases that double as examples of how these functions should work.

In [18]:
def test_lists():
    ls = cons(1, cons(2, cons(3, nil())))
    assert not isEmpty(ls)
    assert head(ls) == 1
    assert head(tail(ls)) == 2
    assert head(tail(tail(ls))) == 3
    assert isEmpty(tail(tail(tail(ls))))

    assert length(ls) == 3
    assert length(tail(ls)) == 2

    assert rev(ls) == cons(3, cons(2, cons(1, nil())))

    assert concat(cons(4, cons(5, nil())), ls) == cons(4, cons(5, cons(1, cons (2, cons (3, nil())))))

test_lists()

# Object-Oriented Linked Lists

It's actually nicer to implement linked lists with Python's *classes*, a core concept of *object-oriented programming*.  It will be natural to adapt our recursive implementations in this setting.  There will be separate classes for empty and nonempty lists.  First, though, we define a *base class* that contains all code common to the two cases, which here will just be the tricky way of implementing concatenation in terms of two reversals.

In [19]:
class List:
    def concat(self, other):
        return self.rev().rev(other)

Now the case for empty lists.  Note that these methods get to be "hardcoded" to work for empty lists only!  That's the magic of object-oriented programming, letting us divide the thinking for functions across several classes that cover different cases.

In [20]:
class Nil(List):
    def toArray(self):
        return []

    def isEmpty(self):
        return True

    def head(self):
        raise ValueError("empty list doesn't have a head")

    def tail(self):
        raise ValueError("empty list doesn't have a tail")

    def length(self):
        return 0

    def rev(self, acc=None):
        if acc == None:
            return self
        else:
            return acc

The methods for nonempty lists are a bit more complex.  Note that an instance of the following class stands for one cell of a linked list, with fields for the head and tail.

In [21]:
class Cons(List):
    def __init__(self, hd, tl):
        self.hd = hd
        self.tl = tl

    def toArray(self):
        return [self.hd] + self.tl.toArray()

    def isEmpty(self):
        return False

    def head(self):
        return self.hd

    def tail(self):
        return self.tl

    def length(self):
        return 1 + self.tl.length()

    def rev(self, acc=Nil()):
        return self.tl.rev(Cons(self.hd, acc))

To repeat our tests from before:

In [22]:
def test_oo_lists():
    ls = Cons(1, Cons(2, Cons(3, Nil())))
    assert not ls.isEmpty()
    assert ls.head() == 1
    assert ls.tail().head() == 2
    assert ls.tail().tail().head() == 3
    assert ls.tail().tail().tail().isEmpty()

    assert ls.length() == 3
    assert ls.tail().length() == 2

    assert ls.rev().toArray() == [3, 2, 1]

    assert Cons(4, Cons(5, Nil())).concat(ls).toArray() == [4, 5, 1, 2, 3]

test_oo_lists()

# Representing an Archive of Course Schedules

The rest of the examples in the lecture code are based on course (subject) schedules for the fictitious Institute of 6.009.  There is likely to be much similarity between schedules across semesters, so we can share plenty of memory contents across many different semesters' schedules.  The basic functionality is a mapping from subject numbers to titles.

We'll work with a class for representing subjects.

In [23]:
class Subject:
    def __init__(self, num, titl):
        self.num = num
        self.titl = titl

    def number(self):
        return self.num

    def title(self):
        return self.titl

Here's one semester's schedule as an array.

In [24]:
initial_schedule = [Subject(100, "Introduction to N-Queens"),
                    Subject(101, "Insights in Image Processing"),
                    Subject(110, "Let's Read the Dictionary"),
                    Subject(111, "Deconstructing Kevin Bacon"),
                    Subject(120, "Readings in Recursion"),
                    Subject(121, "Advanced Minesweeper"),
                    Subject(130, "Sudoku Studies"),
                    Subject(131, "A Brief History of Camping"),
                    Subject(200, "Linked Lists Lab"),
                    Subject(201, "Autocomplete Seminar")]

# A Simple Implementation with Arrays

One of the simplest implementations in Python uses Python lists (arrays), creating a completely distinct schedule for each semester, with no sharing, because we just copy all old entries that we reuse.

In [25]:
class ArraySchedule:
    def __init__(self):
        self.subjects = []

    def lookup(self, subject_number):
        """Return the title of subject in schedule, or None if not found."""

        for subject in self.subjects:
            if subject.number() == subject_number:
                return subject.title()
        return None

    def add(self, subject):
        """Return a new schedule, based on the given one, but with a new entry
        added with given subject and title."""

        new = ArraySchedule()
        new.subjects = self.subjects + [subject]
        return new

# Neat trick: we can write tests generically in a class like this one!
def tests_helper(sched):
    assert sched.lookup(101) == "Insights in Image Processing"

    latest_and_greatest = sched.add(Subject(102, "Tenets of Tent Packing"))
    assert latest_and_greatest.lookup(101) == "Insights in Image Processing"
    assert latest_and_greatest.lookup(102) == "Tenets of Tent Packing"
    assert sched.lookup(102) == None
    # Important: the old schedule is still around and does not contain the new course!

def tests(sched):
    for subject in initial_schedule:
        sched = sched.add(subject)

    tests_helper(sched)

tests(ArraySchedule())

What happens if we create many modified schedules in sequence, and we want to keep all the old ones accessible?  Every schedule uses separate memory space for a different array!  That could be very wasteful as we accumulate more schedules that nonetheless have many similarities.

## Using Linked Lists

An implementation with more sharing uses linked lists.  The simplest variant here allows the schedule entries to occupy arbitrary positions in the list.  For instance, using small integers as subject numbers, we might wind up with a list like this one:

![schedule with linked lists](listsched.png)

To look up a subject in the list, we start at the beginning and follow all the links, checking for subject-number equality until we find the entry or reach the end of the list.  A disadvantage of this representation is that, when a subject does not exist, we must always traverse the list fully before we can be sure.  However, adding a new entry is a simple "cons" operation like before, so we reuse all of the space for the old entries, and the new list can be built more or less instantly, regardless of the schedule size!

In [26]:
class NilSchedule:
    def add(self, subject):
        return ConsSchedule(subject, self)

    def lookup(self, subject_number):
        return None

class ConsSchedule:
    def __init__(self, subject, rest):
        self.hd = subject
        self.tl = rest

    def add(self, subject):
        return ConsSchedule(subject, self)

    def lookup(self, subject_number):
        if self.hd.number() == subject_number:
            return self.hd.title()
        else:
            return self.tl.lookup(subject_number)

tests(NilSchedule())

## Sorted Linked Lists

We can keep much of the last solution's benefit and dodge the first complaint (having to traverse the whole list to realize a subject is missing) by maintaining the list in sorted order, more like this:

![schedule with sorted linked lists](sortedlistsched.png)

Now, in some rough sense, lookups take “about half as long,” on average.  Unfortunately, adding a new element now requires more work, to maintain sorting.  We often need to recreate copies of existing list cells.

In [27]:
class NilSortedSchedule:
    def add(self, subject):
        return ConsSortedSchedule(subject, self)

    def lookup(self, subject_number):
        return None

class ConsSortedSchedule:
    def __init__(self, subject, rest):
        self.hd = subject
        self.tl = rest

    def add(self, subject):
        if subject.number() < self.hd.number():
            return ConsSortedSchedule(subject, self)
        else:
            return ConsSortedSchedule(self.hd, self.tl.add(subject))

    def lookup(self, subject_number):
        if self.hd.number() == subject_number:
            return self.hd.title()
        elif self.hd.number() > subject_number:
            return None
        else:
            return self.tl.lookup(subject_number)

tests(NilSortedSchedule())

## Binary Search Trees (a true classic data structure)

We can do even better than cutting lookup time by “about half.”  Instead we can get it to be “about the logarithm of the schedule size,” while still maintaining sharing across versions.  (We're being fuzzy about timing details here, saving the specifics for 6.006 and its follow-up courses.)

Our new approach is *binary search trees*, which look like this:

![a binary search tree](bst.png)

The top node of the tree we call the *root*.  Here we have a good opportunity to introduce one of the oddities of programming terminology: in the world of computer science, trees grow upside down, with their roots on top!  Each node may have some *children*, connected with arrows going down.  Each node is also labeled with the key associated with it.  Notice that, reading left to right, we encounter the keys in sorted order.

The basic idea of binary search trees is that each node is implicitly associated with a range of keys that could possibly be found there.  We use the same interval notation that you probably know from calculus class.  In any kind of lookup, we can skip whole subtrees whose intervals couldn't possibly contain the key we care about!  That's the key to speeding up all operations, as compared to sorted lists.

The root of the tree is associated with interval \[2, 7], containing all keys from the tree.  The root includes key 5.  All keys *less than 5 must be in its left child*, and all keys *greater than 5 must be in its right child*.  We illustrate this rule in tagging nodes with their intervals.  “Zigging left” pulls down the high end of the interval to equal the value we've skipped, and “zagging right” pushes up the low end of the interval to equal the value we've skipped.

When searching for a key, we recursively walk down the tree.  If the key on the node matches the one we want, we're done.  Otherwise, we know *exactly which child tree to search next*, because their intervals have no keys in common!  We get to skip entirely the irrelevant subtree.

Let's make all that concrete for our running example of the subject schedule.

In [28]:
class EmptyTreeSchedule:
    def add(self, subject):
        return NonemptyTreeSchedule(subject,
                                    EmptyTreeSchedule(),
                                    EmptyTreeSchedule())

    def lookup(self, subject_number):
        return None

class NonemptyTreeSchedule:
    def __init__(self, subject, leftChild, rightChild):
        self.subject = subject
        self.leftChild = leftChild
        self.rightChild = rightChild

    def add(self, subject):
        if subject.number() == self.subject.number():
            # In case of a subject-number match, we *overwrite* the root.
            return NonemptyTreeSchedule(subject,
                                        self.leftChild,
                                        self.rightChild)
        elif subject.number() < self.subject.number():
            return NonemptyTreeSchedule(self.subject,
                                        self.leftChild.add(subject),
                                        self.rightChild)
        else:
            return NonemptyTreeSchedule(self.subject,
                                        self.leftChild,
                                        self.rightChild.add(subject))

    def lookup(self, subject_number):
        if subject_number == self.subject.number():
            return self.subject.title()
        elif subject_number < self.subject.number():
            return self.leftChild.lookup(subject_number)
        else:
            return self.rightChild.lookup(subject_number)

tests(EmptyTreeSchedule())

### A Performance Gotcha

Binary search trees manage to reuse many tree nodes across many different versions.  However, the way we build the tree schedule in the tests is actually rather inefficient!

In [29]:
import sys
sys.setrecursionlimit(10000)

tree = EmptyTreeSchedule()
for i in range(3000):
    tree = tree.add(Subject(i, "Redundancy"))
tree.lookup(2500)

'Redundancy'

Pretty slow, right?  What happened is that we actually wound up with degenerate trees that act like sorted linked lists!  Trace through the algorithm (on a smaller example) to see for yourself.

Here's a function that can help us translate to trees more efficiently, when we know we're starting with a sorted array.  The key is to adopt a divide-and-conquer strategy with recursion.

In [30]:
def sorted_array_to_bst(arr):
    if len(arr) == 0:
        return EmptyTreeSchedule()
    else:
        # The array has at least two elements.
        # Pick some spot in the middle and build a tree for either side of it.
        # Then create a new tree root from the middle element.
        mid = len(arr) // 2
        return NonemptyTreeSchedule(arr[mid],
                                    sorted_array_to_bst(arr[:mid]),
                                    sorted_array_to_bst(arr[mid+1:]))

tests_helper(sorted_array_to_bst(initial_schedule))

tree = sorted_array_to_bst([Subject(i, "Redundancy")
                            for i in range(3000)])
tree.lookup(2500)

'Redundancy'

...and now the lookup is instant!  By the way, see 6.006 for fancier tree structures that can provide fast lookups regardless of order of insertion.

# Sudoku Revisited

Here's a literal copy of the most optimized Sudoku code from the previous lecture.

In [31]:
backtracks = 0

#x varies from entry1 to entry2 - 1, y varies from entry3 to entry4 - 1 
sectors = [ [0, 3, 0, 3], [3, 6, 0, 3], [6, 9, 0, 3],
            [0, 3, 3, 6], [3, 6, 3, 6], [6, 9, 3, 6],
            [0, 3, 6, 9], [3, 6, 6, 9], [6, 9, 6, 9] ]

#This procedure finds the next empty square to fill on the Sudoku grid
def findNextCellToFill(grid):
    #Look for an unfilled grid location
    for x in range(0, 9):
        for y in range(0, 9):
            if grid[x][y] == 0:
                return x,y
    return -1,-1

#This procedure checks if setting the (i, j) square to e is valid
def isValid(grid, i, j, e):
    rowOk = all([e != grid[i][x] for x in range(9)])
    if rowOk:
        columnOk = all([e != grid[x][j] for x in range(9)])
        if columnOk:
            #finding the top left x,y co-ordinates of
            #the section or sub-grid containing the i,j cell
            secTopX, secTopY = 3 *(i//3), 3 *(j//3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if grid[x][y] == e:
                        return False
            return True
    return False

#This procedure makes implications based on existing numbers on squares
def makeImplications(grid, i, j, e):

    global sectors

    grid[i][j] = e
    impl = [(i, j, e)]

    done = False

    #Keep going till you stop finding implications
    while not done:
        done = True

        for k in range(len(sectors)):

            sectinfo = []

            #find missing elements in ith sector
            vset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
                    if grid[x][y] != 0:
                        vset.remove(grid[x][y])

            #attach copy of vset to each missing square in ith sector
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
                    if grid[x][y] == 0:
                        sectinfo.append([x, y, vset.copy()])
            
            for m in range(len(sectinfo)):
                sin = sectinfo[m]
                
                #find the set of elements on the row corresponding to m and remove them
                rowv = set()
                for y in range(9):
                    rowv.add(grid[sin[0]][y])
                left = sin[2].difference(rowv)
                
                #find the set of elements on the column corresponding to m and remove them
                colv = set()
                for x in range(9):
                    colv.add(grid[x][sin[1]])
                left = left.difference(colv)
                             
                #check if the vset is a singleton
                if len(left) == 1:
                    val = left.pop()
                    if isValid(grid, sin[0], sin[1], val):
                        grid[sin[0]][sin[1]] = val
                        impl.append((sin[0], sin[1], val))
                        done = False
                
    return impl

#This procedure undoes all the implications
def undoImplications(grid, impl):
    for i in range(len(impl)):
        grid[impl[i][0]][impl[i][1]] = 0
    return

#This procedure fills in the missing squares of a Sudoku puzzle
#obeying the Sudoku rules by guessing when it has to and performing
#implications when it can
def solveSudokuMoreOpt(grid, i=0, j=0):

    global backtracks

    #find the next empty cell to fill
    i, j = findNextCellToFill(grid)
    if i == -1:
        return True

    for e in range(1, 10):
        #Try different values in i, j location
        if isValid(grid, i, j, e):

            impl = makeImplications(grid, i, j, e)
            
            if solveSudokuMoreOpt(grid, i, j):
                return True
            #Undo the current cell for backtracking
            backtracks += 1
            undoImplications(grid, impl)

    return False

def printSudoku(grid):
    numrow = 0
    for row in grid:
        if numrow % 3 == 0 and numrow != 0:
            print (' ')
        print (row[0:3], ' ', row[3:6], ' ', row[6:9])
        numrow += 1       
    return

diff  = [[0,0,5,3,0,0,0,0,0],
         [8,0,0,0,0,0,0,2,0],
         [0,7,0,0,1,0,5,0,0],
         [4,0,0,0,0,5,3,0,0],
         [0,1,0,0,7,0,0,0,6],
         [0,0,3,2,0,0,0,8,0],
         [0,6,0,5,0,0,0,0,9],
         [0,0,4,0,0,0,0,3,0],
         [0,0,0,0,0,9,7,0,0]]

backtracks = 0
solveSudokuMoreOpt(diff)
print ('Backtracks =', backtracks)

Backtracks = 232


Remember all the work we did to track changes made to the board, so we could undo them when backtracking?  How about if we represent boards as *binary search trees* whose keys are coordinates?  Then we can maintain multiple boards at once, with significant sharing across them!  We'll do it in a slightly gross way, reusing our subject-scheduling trees, representing each board square as a subject whose number is its coordinates and name is the value of the square (e.g., a digit).

Here's the code again following that strategy, where comments like `# CHANGE! --->` indicate where we made modifications.

In [32]:
#This procedure finds the next empty square to fill on the Sudoku grid
def findNextCellToFill(grid):
    #Look for an unfilled grid location
    for x in range(0, 9):
        for y in range(0, 9):
# CHANGE! --->
# Look up grid cell with a method, by coordinates.
            if grid.lookup((x, y)) == 0:
                return x,y
    return -1,-1

#This procedure checks if setting the (i, j) square to e is valid
def isValid(grid, i, j, e):
# CHANGE! --->
# Look up by coordinates.
    rowOk = all([e != grid.lookup((i, x)) for x in range(9)])
    if rowOk:
        columnOk = all([e != grid.lookup((x, j)) for x in range(9)])
        if columnOk:
            #finding the top left x,y co-ordinates of
            #the section or sub-grid containing the i,j cell
            secTopX, secTopY = 3 *(i//3), 3 *(j//3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if grid.lookup((x, y)) == e:
                        return False
            return True
    return False

#This procedure makes implications based on existing numbers on squares
def makeImplications(grid, i, j, e):

    global sectors

# CHANGE! --->
# Generate a _new_ grid without overwriting the old one.
# (For simplicity, we'll use the same variable to hold the new one,
# since we never need to reference the old version again in this function.)
    grid = grid.add(Subject((i, j), e))
# CHANGE! --->
# We no longer track a list of implications.
#   impl = [(i, j, e)]

    done = False

    #Keep going till you stop finding implications
    while not done:
        done = True

        for k in range(len(sectors)):

            sectinfo = []

            #find missing elements in ith sector
            vset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
# CHANGE! --->
# Look up by coordinates.
                    if grid.lookup((x, y)) != 0:
                        vset.remove(grid.lookup((x, y)))

            #attach copy of vset to each missing square in ith sector
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
# CHANGE! --->
# Look up by coordinates.
                    if grid.lookup((x,y)) == 0:
                        sectinfo.append([x, y, vset.copy()])
            
            for m in range(len(sectinfo)):
                sin = sectinfo[m]
                
                #find the set of elements on the row corresponding to m and remove them
                rowv = set()
                for y in range(9):
# CHANGE! --->
# Look up by coordinates.
                    rowv.add(grid.lookup((sin[0], y)))
                left = sin[2].difference(rowv)
                
                #find the set of elements on the column corresponding to m and remove them
                colv = set()
                for x in range(9):
# CHANGE! --->
# Look up by coordinates.
                    colv.add(grid.lookup((x, sin[1])))
                left = left.difference(colv)
                             
                #check if the vset is a singleton
                if len(left) == 1:
                    val = left.pop()
                    if isValid(grid, sin[0], sin[1], val):
# CHANGE! --->
# Set by coordinates, overwriting grid variable.
                        grid = grid.add(Subject((sin[0], sin[1]), val))
# CHANGE! --->
# Not maintaining impl anymore.
#                       impl.append((sin[0], sin[1], val))
                        done = False

# CHANGE! --->
# Return new grid, rather than implications.
    return grid

# CHANGE! --->
# We don't need an undoing procedure anymore!
#This procedure undoes all the implications
#def undoImplications(grid, impl):
#    for i in range(len(impl)):
#        grid[impl[i][0]][impl[i][1]] = 0
#    return

# CHANGE! --->
# Converting into our funky board format.
def convert_board(grid, empty):
    new_grid = empty
    for i in range(9):
        for j in range(9):
            new_grid = new_grid.add(Subject((i, j), grid[i][j]))
    return new_grid

#This procedure fills in the missing squares of a Sudoku puzzle
#obeying the Sudoku rules by guessing when it has to and performing
#implications when it can
def solveSudokuMoreOpt(grid, i=0, j=0):

    global backtracks
    
    #find the next empty cell to fill
    i, j = findNextCellToFill(grid)
    if i == -1:
        return True

    for e in range(1, 10):
        #Try different values in i, j location
        if isValid(grid, i, j, e):
# CHANGE! --->
# Save new grid in a separate variable here.
            new_grid = makeImplications(grid, i, j, e)
            
            if solveSudokuMoreOpt(new_grid, i, j):
                return True
            #Undo the current cell for backtracking
            backtracks += 1
# CHANGE! --->
# No undoing required; just go back to using the old value!
#           undoImplications(grid, impl)

    return False

diff  = [[0,0,5,3,0,0,0,0,0],
         [8,0,0,0,0,0,0,2,0],
         [0,7,0,0,1,0,5,0,0],
         [4,0,0,0,0,5,3,0,0],
         [0,1,0,0,7,0,0,0,6],
         [0,0,3,2,0,0,0,8,0],
         [0,6,0,5,0,0,0,0,9],
         [0,0,4,0,0,0,0,3,0],
         [0,0,0,0,0,9,7,0,0]]

backtracks = 0
solveSudokuMoreOpt(convert_board(diff, EmptyTreeSchedule()))
print ('Backtracks =', backtracks)

Backtracks = 232


Notice that this version is slower than the undoing version, but one could argue we saved in code understandability.